In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


class display(object):
    """Display HTML representation of multiple objects"""

    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""

    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        return '\n'.join(
            self.template.format(a, eval(a)._repr_html_()) for a in self.args
        )

    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a)) for a in self.args)



In [2]:
import numpy as np

a = np.array([[1, 2, 3], [4, 5, 6]])
a

b = a.flatten()
b
c = a.flatten('F')
c

array([[1, 2, 3],
       [4, 5, 6]])

array([1, 2, 3, 4, 5, 6])

array([1, 4, 2, 5, 3, 6])

In [3]:
from dataclasses import dataclass, field
from typing import Optional
import pandas as pd
import numpy as np
from ordered_set import OrderedSet
from eos.data_io.config import drivers, trucks_by_name, drivers_by_id

# DriverSet = OrderedSet(
#     [
#         'wang-cheng',
#         'li-changlong',
#         'chen-hongmei',
#         'zheng-longfei',
#     ]
# )
# DriverSet[0]
# DriverSet.get_loc('wang-cheng')
# DriverSet


Pool.__init_subclass__(): ~DocItemT; Concrete Class: (eos.data_io.pool.pool.Pool[~DocItemT], typing.Generic[~DocItemT])
Pool.__init_subclass__(): <class 'eos.data_io.struct.eos_data.RecordDoc'>; Concrete Class: (eos.data_io.pool.pool.Pool[eos.data_io.struct.eos_data.RecordDoc],)
Pool.__init_subclass__(): ~DocItemT; Concrete Class: (eos.data_io.pool.pool.Pool[~DocItemT], typing.Generic[~DocItemT])
Pool.__init_subclass__(): ~DocItemT; Concrete Class: (eos.data_io.pool.pool.Pool[~DocItemT], typing.Generic[~DocItemT])
Pool.__init_subclass__(): ~DocItemT; Concrete Class: (eos.data_io.buffer.buffer.Buffer[~DocItemT], typing.Generic[~DocItemT])
Pool.__init_subclass__(): ~DocItemT; Concrete Class: (eos.data_io.buffer.buffer.Buffer[~DocItemT], typing.Generic[~DocItemT])


2023-06-12 17:03:02.445859: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
import pandas as pd
import numpy as np
from datetime import datetime

ts = pd.to_datetime(datetime.now())
ts_ind = ts + pd.to_timedelta(np.arange(2), 'H')
ts_ind
df = pd.DataFrame(a, index=ts_ind, columns=['c1', 'c2', 'c3'])
df
df.index[0]


DatetimeIndex(['2023-06-12 17:03:03.583607', '2023-06-12 18:03:03.583607'], dtype='datetime64[ns]', freq=None)

,c1,c2,c3
2023-06-12 17:03:03.583607,1,2,3
2023-06-12 18:03:03.583607,4,5,6


Timestamp('2023-06-12 17:03:03.583607')

In [5]:
df_ts = pd.DataFrame(np.array([ts]), columns=['timestamp'])
df_ts

,timestamp
0,2023-06-12 17:03:03.583607


In [6]:
s = np.arange(10)
s

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

### Generate state

In [7]:
ss = np.arange(12)
a1 = ss[:4].tolist()
a2 = ss[4:8].tolist()
a3 = ss[8:].tolist()
ss = [a1, a2, a3]
ss
df_s = pd.DataFrame(ss, columns=['t', 'v', 'p', 'b'])  # .set_index('t')
df_s

[[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11]]

,t,v,p,b
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


In [8]:
np.arange(0, 4 * 20, 20)

array([ 0, 20, 40, 60])

In [31]:
ts_ind = ts + pd.to_timedelta(np.arange(0, 4 * 20, 20), 'ms')
ts_ind
ss = np.arange(12)
a1 = ss[:4]
a2 = ss[4:8]
a3 = ss[8:]
df_ss = pd.DataFrame(
    {'timestep': ts_ind, 'velocity': a1, 'thrust': a2, 'brake': a3}
).set_index('timestep')
df_ss.columns.name = 'tuple'
df_ss


DatetimeIndex(['2023-06-12 17:03:03.583607', '2023-06-12 17:03:03.603607',
               '2023-06-12 17:03:03.623607', '2023-06-12 17:03:03.643607'],
              dtype='datetime64[ns]', freq=None)

tuple,velocity,thrust,brake
timestep,,,
2023-06-12 17:03:03.583607,0,4,8
2023-06-12 17:03:03.603607,1,5,9
2023-06-12 17:03:03.623607,2,6,10
2023-06-12 17:03:03.643607,3,7,11


In [32]:
state = df_ss.stack().swaplevel(0, 1)
state.name = 'state'
state.sort_index(inplace=True)
state

tuple     timestep                  
brake     2023-06-12 17:03:03.583607     8
          2023-06-12 17:03:03.603607     9
          2023-06-12 17:03:03.623607    10
          2023-06-12 17:03:03.643607    11
thrust    2023-06-12 17:03:03.583607     4
          2023-06-12 17:03:03.603607     5
          2023-06-12 17:03:03.623607     6
          2023-06-12 17:03:03.643607     7
velocity  2023-06-12 17:03:03.583607     0
          2023-06-12 17:03:03.603607     1
          2023-06-12 17:03:03.623607     2
          2023-06-12 17:03:03.643607     3
Name: state, dtype: int64

In [11]:
# from functools import reduce
#
#
# velocity = pd.Series(ss[:4], name='velocity')
# thrust = pd.Series(ss[4:8], name='thrust')
# brake = pd.Series(ss[8:], name='brake')
# series = [velocity, thrust, brake]
# state = reduce(
#     lambda x, y: pd.merge(x, y, how='outer', left_index=True, right_index=True), series
# )
# state = state.stack().swaplevel(0, 1).sort_index()
# state.name = 'state'
# state
# len(state)


In [12]:
# from functools import reduce
#
#
# velocity = pd.Series(s[:4], name='velocity')
# thrust = pd.Series(s[4:7], name='thrust')
# brake = pd.Series(s[7:], name='brake')
# series = [velocity, thrust, brake]
# state = reduce(
#     lambda x, y: pd.merge(x, y, how='outer', left_index=True, right_index=True), series
# )
# state = state.stack().swaplevel(0, 1).sort_index()
# state.name = 'state'
# state
# len(state)
#
#

In [33]:
row_ind = np.arange(50, 80, 10)
# row_ind
ts_ind = ts + pd.to_timedelta(np.arange(5 * 20, 8 * 20, 20), 'ms')
pedal_ind = np.linspace(0, 1.0, 5)
# pedal_ind
# action_ind = pd.MultiIndex.from_product([row_ind, ts_ind], names=['row', 'timestamp'])
a = len(state) + np.arange(15)
# a
action = (
    pd.DataFrame(
        {
            'pedal': pedal_ind,
            row_ind[0]: a[:5],
            row_ind[1]: a[5:10],
            row_ind[2]: a[10:],
        }
    ).set_index('pedal')
    # .stack()
    # .swaplevel(0, 1)
    .sort_index()
)
action.columns
action.columns = pd.MultiIndex.from_arrays(
    [action.columns, ts_ind], names=['nominalspeed', 'timestep']
)
action = action.stack([0, 1]).swaplevel(0, 1).swaplevel(1, 2).sort_index()
action.name = 'action'
# action.index.names = ['nominalspeed', 'timestamp']
# action.index
action

Index([50, 60, 70], dtype='object')

nominalspeed  timestep                    pedal
50            2023-06-12 17:03:03.683607  0.00     12.0
                                          0.25     13.0
                                          0.50     14.0
                                          0.75     15.0
                                          1.00     16.0
60            2023-06-12 17:03:03.703607  0.00     17.0
                                          0.25     18.0
                                          0.50     19.0
                                          0.75     20.0
                                          1.00     21.0
70            2023-06-12 17:03:03.723607  0.00     22.0
                                          0.25     23.0
                                          0.50     24.0
                                          0.75     25.0
                                          1.00     26.0
Name: action, dtype: float64

In [14]:
# a = len(state) + np.arange(6)
# ar0 = pd.Series(a[:2], name='row0')
# ar1 = pd.Series(a[2:4], name='row1')
# ar2 = pd.Series(a[4:], name='row2')
# series = [ar0, ar1, ar2]
# action = reduce(
#     lambda x, y: pd.merge(x, y, how='outer', left_index=True, right_index=True), series
# )
# action = action.stack().swaplevel(0, 1).sort_index()
# action.name = 'action'
# action
# len(action) + len(state)

In [15]:
reward = pd.Series([len(state) + len(action)], name='reward')
# reward.name = 'reward'

In [34]:
# velocity = pd.Series(len(state) + len(action) + len(reward) + s[:4], name='velocity')
# thrust = pd.Series(len(state) + len(action) + len(reward) + s[4:7], name='thrust')
# brake = pd.Series(len(state) + len(action) + len(reward) + s[7:], name='brake')
# series = [velocity, thrust, brake]
# nstate = reduce(
#     lambda x, y: pd.merge(x, y, how='outer', left_index=True, right_index=True), series
# )
# nstate = nstate.stack().swaplevel(0, 1).sort_index()
# nstate.name = 'nstate'
# nstate
# len(nstate)
ts_ind = ts + pd.to_timedelta(5, 's') + pd.to_timedelta(np.arange(0, 4 * 20, 20), 'ms')
ts_ind
ss = np.arange(12) + len(state) + len(action) + len(reward)
a1 = ss[:4]
a2 = ss[4:8]
a3 = ss[8:]
nstate = (
    pd.DataFrame({'timestamp': ts_ind, 'velocity': a1, 'thrust': a2, 'brake': a3})
    .set_index('timestamp')
    .stack()
    .swaplevel(0, 1)
    .sort_index()
)
nstate.name = 'nstate'
nstate.index.names = ['tuple', 'timestep']
len(nstate)
nstate

DatetimeIndex(['2023-06-12 17:03:08.583607', '2023-06-12 17:03:08.603607',
               '2023-06-12 17:03:08.623607', '2023-06-12 17:03:08.643607'],
              dtype='datetime64[ns]', freq=None)

12

tuple     timestep                  
brake     2023-06-12 17:03:08.583607    36
          2023-06-12 17:03:08.603607    37
          2023-06-12 17:03:08.623607    38
          2023-06-12 17:03:08.643607    39
thrust    2023-06-12 17:03:08.583607    32
          2023-06-12 17:03:08.603607    33
          2023-06-12 17:03:08.623607    34
          2023-06-12 17:03:08.643607    35
velocity  2023-06-12 17:03:08.583607    28
          2023-06-12 17:03:08.603607    29
          2023-06-12 17:03:08.623607    30
          2023-06-12 17:03:08.643607    31
Name: nstate, dtype: int64

In [35]:
timestamp = pd.Series([ts], name='timestamp')
timestamp.index
state.index
action.index
reward.index
nstate.index

RangeIndex(start=0, stop=1, step=1)

MultiIndex([(   'brake', '2023-06-12 17:03:03.583607'),
            (   'brake', '2023-06-12 17:03:03.603607'),
            (   'brake', '2023-06-12 17:03:03.623607'),
            (   'brake', '2023-06-12 17:03:03.643607'),
            (  'thrust', '2023-06-12 17:03:03.583607'),
            (  'thrust', '2023-06-12 17:03:03.603607'),
            (  'thrust', '2023-06-12 17:03:03.623607'),
            (  'thrust', '2023-06-12 17:03:03.643607'),
            ('velocity', '2023-06-12 17:03:03.583607'),
            ('velocity', '2023-06-12 17:03:03.603607'),
            ('velocity', '2023-06-12 17:03:03.623607'),
            ('velocity', '2023-06-12 17:03:03.643607')],
           names=['tuple', 'timestep'])

MultiIndex([(50, '2023-06-12 17:03:03.683607',  0.0),
            (50, '2023-06-12 17:03:03.683607', 0.25),
            (50, '2023-06-12 17:03:03.683607',  0.5),
            (50, '2023-06-12 17:03:03.683607', 0.75),
            (50, '2023-06-12 17:03:03.683607',  1.0),
            (60, '2023-06-12 17:03:03.703607',  0.0),
            (60, '2023-06-12 17:03:03.703607', 0.25),
            (60, '2023-06-12 17:03:03.703607',  0.5),
            (60, '2023-06-12 17:03:03.703607', 0.75),
            (60, '2023-06-12 17:03:03.703607',  1.0),
            (70, '2023-06-12 17:03:03.723607',  0.0),
            (70, '2023-06-12 17:03:03.723607', 0.25),
            (70, '2023-06-12 17:03:03.723607',  0.5),
            (70, '2023-06-12 17:03:03.723607', 0.75),
            (70, '2023-06-12 17:03:03.723607',  1.0)],
           names=['nominalspeed', 'timestep', 'pedal'])

RangeIndex(start=0, stop=1, step=1)

MultiIndex([(   'brake', '2023-06-12 17:03:08.583607'),
            (   'brake', '2023-06-12 17:03:08.603607'),
            (   'brake', '2023-06-12 17:03:08.623607'),
            (   'brake', '2023-06-12 17:03:08.643607'),
            (  'thrust', '2023-06-12 17:03:08.583607'),
            (  'thrust', '2023-06-12 17:03:08.603607'),
            (  'thrust', '2023-06-12 17:03:08.623607'),
            (  'thrust', '2023-06-12 17:03:08.643607'),
            ('velocity', '2023-06-12 17:03:08.583607'),
            ('velocity', '2023-06-12 17:03:08.603607'),
            ('velocity', '2023-06-12 17:03:08.623607'),
            ('velocity', '2023-06-12 17:03:08.643607')],
           names=['tuple', 'timestep'])

In [36]:
timestamp_index = (timestamp.name, '', 0, '')
timestamp_index
state_index = [(state.name, *i, '') for i in state.index]
state_index
reward_index = (reward.name, '', 0, '')
reward_index
action_index = [(action.name, *i) for i in action.index]
action_index
nstate_index = [(nstate.name, *i, '') for i in nstate.index]
nstate_index

multiindex = pd.MultiIndex.from_tuples(
    [timestamp_index, *state_index, reward_index, *action_index, *nstate_index]
)
multiindex


('timestamp', '', 0, '')

[('state', 'brake', Timestamp('2023-06-12 17:03:03.583607'), ''),
 ('state', 'brake', Timestamp('2023-06-12 17:03:03.603607'), ''),
 ('state', 'brake', Timestamp('2023-06-12 17:03:03.623607'), ''),
 ('state', 'brake', Timestamp('2023-06-12 17:03:03.643607'), ''),
 ('state', 'thrust', Timestamp('2023-06-12 17:03:03.583607'), ''),
 ('state', 'thrust', Timestamp('2023-06-12 17:03:03.603607'), ''),
 ('state', 'thrust', Timestamp('2023-06-12 17:03:03.623607'), ''),
 ('state', 'thrust', Timestamp('2023-06-12 17:03:03.643607'), ''),
 ('state', 'velocity', Timestamp('2023-06-12 17:03:03.583607'), ''),
 ('state', 'velocity', Timestamp('2023-06-12 17:03:03.603607'), ''),
 ('state', 'velocity', Timestamp('2023-06-12 17:03:03.623607'), ''),
 ('state', 'velocity', Timestamp('2023-06-12 17:03:03.643607'), '')]

('reward', '', 0, '')

[('action', 50, Timestamp('2023-06-12 17:03:03.683607'), 0.0),
 ('action', 50, Timestamp('2023-06-12 17:03:03.683607'), 0.25),
 ('action', 50, Timestamp('2023-06-12 17:03:03.683607'), 0.5),
 ('action', 50, Timestamp('2023-06-12 17:03:03.683607'), 0.75),
 ('action', 50, Timestamp('2023-06-12 17:03:03.683607'), 1.0),
 ('action', 60, Timestamp('2023-06-12 17:03:03.703607'), 0.0),
 ('action', 60, Timestamp('2023-06-12 17:03:03.703607'), 0.25),
 ('action', 60, Timestamp('2023-06-12 17:03:03.703607'), 0.5),
 ('action', 60, Timestamp('2023-06-12 17:03:03.703607'), 0.75),
 ('action', 60, Timestamp('2023-06-12 17:03:03.703607'), 1.0),
 ('action', 70, Timestamp('2023-06-12 17:03:03.723607'), 0.0),
 ('action', 70, Timestamp('2023-06-12 17:03:03.723607'), 0.25),
 ('action', 70, Timestamp('2023-06-12 17:03:03.723607'), 0.5),
 ('action', 70, Timestamp('2023-06-12 17:03:03.723607'), 0.75),
 ('action', 70, Timestamp('2023-06-12 17:03:03.723607'), 1.0)]

[('nstate', 'brake', Timestamp('2023-06-12 17:03:08.583607'), ''),
 ('nstate', 'brake', Timestamp('2023-06-12 17:03:08.603607'), ''),
 ('nstate', 'brake', Timestamp('2023-06-12 17:03:08.623607'), ''),
 ('nstate', 'brake', Timestamp('2023-06-12 17:03:08.643607'), ''),
 ('nstate', 'thrust', Timestamp('2023-06-12 17:03:08.583607'), ''),
 ('nstate', 'thrust', Timestamp('2023-06-12 17:03:08.603607'), ''),
 ('nstate', 'thrust', Timestamp('2023-06-12 17:03:08.623607'), ''),
 ('nstate', 'thrust', Timestamp('2023-06-12 17:03:08.643607'), ''),
 ('nstate', 'velocity', Timestamp('2023-06-12 17:03:08.583607'), ''),
 ('nstate', 'velocity', Timestamp('2023-06-12 17:03:08.603607'), ''),
 ('nstate', 'velocity', Timestamp('2023-06-12 17:03:08.623607'), ''),
 ('nstate', 'velocity', Timestamp('2023-06-12 17:03:08.643607'), '')]

MultiIndex([('timestamp',         '',                          0,   ''),
            (    'state',    'brake', 2023-06-12 17:03:03.583607,   ''),
            (    'state',    'brake', 2023-06-12 17:03:03.603607,   ''),
            (    'state',    'brake', 2023-06-12 17:03:03.623607,   ''),
            (    'state',    'brake', 2023-06-12 17:03:03.643607,   ''),
            (    'state',   'thrust', 2023-06-12 17:03:03.583607,   ''),
            (    'state',   'thrust', 2023-06-12 17:03:03.603607,   ''),
            (    'state',   'thrust', 2023-06-12 17:03:03.623607,   ''),
            (    'state',   'thrust', 2023-06-12 17:03:03.643607,   ''),
            (    'state', 'velocity', 2023-06-12 17:03:03.583607,   ''),
            (    'state', 'velocity', 2023-06-12 17:03:03.603607,   ''),
            (    'state', 'velocity', 2023-06-12 17:03:03.623607,   ''),
            (    'state', 'velocity', 2023-06-12 17:03:03.643607,   ''),
            (   'reward',         '',              

In [37]:
observation_list = [timestamp, state, action, reward, nstate]
observation = pd.concat(observation_list)
observation.index = multiindex
observation
observation.dtype

timestamp            0                                   2023-06-12 17:03:03.583607
state      brake     2023-06-12 17:03:03.583607                                   8
                     2023-06-12 17:03:03.603607                                   9
                     2023-06-12 17:03:03.623607                                  10
                     2023-06-12 17:03:03.643607                                  11
           thrust    2023-06-12 17:03:03.583607                                   4
                     2023-06-12 17:03:03.603607                                   5
                     2023-06-12 17:03:03.623607                                   6
                     2023-06-12 17:03:03.643607                                   7
           velocity  2023-06-12 17:03:03.583607                                   0
                     2023-06-12 17:03:03.603607                                   1
                     2023-06-12 17:03:03.623607                             

dtype('O')

In [38]:

observation0 = observation.copy()
observation0.loc['timestamp', '', 0, ''] = ts + pd.Timedelta(1, 'h')
observation1 = observation.copy()
observation1.loc['timestamp', '', 0, ''] = ts + pd.Timedelta(2, 'h')
observation2 = observation.copy()
observation2.loc['timestamp', '', 0, ''] = ts + pd.Timedelta(3, 'h')
observation3 = observation.copy()
observation3.loc['timestamp', '', 0, ''] = ts + pd.Timedelta(4, 'h')
observation4 = observation.copy()
observation4.loc['timestamp', '', 0, ''] = ts + pd.Timedelta(5, 'h')
observation_list = [
    observation0,
    observation1,
    observation2,
    observation3,
    observation4,
]


In [41]:

dfs_epi = pd.concat(observation_list, axis=1).transpose()
dfs_epi.columns.names = ['tuple', 'measurement', 'timestep', 'extra']
dfs_epi.columns
display('dfs_epi')


MultiIndex([('timestamp',         '',                          0,   ''),
            (    'state',    'brake', 2023-06-12 17:03:03.583607,   ''),
            (    'state',    'brake', 2023-06-12 17:03:03.603607,   ''),
            (    'state',    'brake', 2023-06-12 17:03:03.623607,   ''),
            (    'state',    'brake', 2023-06-12 17:03:03.643607,   ''),
            (    'state',   'thrust', 2023-06-12 17:03:03.583607,   ''),
            (    'state',   'thrust', 2023-06-12 17:03:03.603607,   ''),
            (    'state',   'thrust', 2023-06-12 17:03:03.623607,   ''),
            (    'state',   'thrust', 2023-06-12 17:03:03.643607,   ''),
            (    'state', 'velocity', 2023-06-12 17:03:03.583607,   ''),
            (    'state', 'velocity', 2023-06-12 17:03:03.603607,   ''),
            (    'state', 'velocity', 2023-06-12 17:03:03.623607,   ''),
            (    'state', 'velocity', 2023-06-12 17:03:03.643607,   ''),
            (   'reward',         '',              

dfs_epi
tuple                        timestamp                      state   
measurement                                                 brake   
timestep                             0 2023-06-12 17:03:03.583607   
extra                                                               
0           2023-06-12 18:03:03.583607                          8  \
1           2023-06-12 19:03:03.583607                          8   
2           2023-06-12 20:03:03.583607                          8   
3           2023-06-12 21:03:03.583607                          8   
4           2023-06-12 22:03:03.583607                          8   

tuple                                                               
measurement                                                         
timestep    2023-06-12 17:03:03.603607 2023-06-12 17:03:03.623607   
extra                                                               
0                                    9                         10  \
1                                    9                         10   
2                                    9                         10   
3                                    9                         10   
4                                    9                         10   

tuple                                                               
measurement                                                thrust   
timestep    2023-06-12 17:03:03.643607 2023-06-12 17:03:03.583607   
extra                                                               
0                                   11                          4  \
1                                   11                          4   
2                                   11                          4   
3                                   11                          4   
4                                   11                          4   

tuple                                                               
measurement                                                         
timestep    2023-06-12 17:03:03.603607 2023-06-12 17:03:03.623607   
extra                                                               
0                                    5                          6  \
1                                    5                          6   
2                                    5                          6   
3                                    5                          6   
4                                    5                          6   

tuple                                                              ...   
measurement                                              velocity  ...   
timestep    2023-06-12 17:03:03.643607 2023-06-12 17:03:03.583607  ...   
extra                                                              ...   
0                                    7                          0  ...  \
1                                    7                          0  ...   
2                                    7                          0  ...   
3                                    7                          0  ...   
4                                    7                          0  ...   

tuple                           nstate                              
measurement                      brake                              
timestep    2023-06-12 17:03:08.623607 2023-06-12 17:03:08.643607   
extra                                                               
0                                   38                         39  \
1                                   38                         39   
2                                   38                         39   
3                                   38                         39   
4                                   38                         39   

tuple                                                               
measurement                     thrust                              
timestep    2023-06-12 17:03:08.583607 2023-06-12 17:03:08.603607   
extra   

In [42]:
dfs_episode1 = dfs_epi.copy().sort_index(axis=1)
dfs_episode1.set_index(('timestamp', '', 0, ''), inplace=True)
dfs_episode1.index.name = 'timestamp'
idx = pd.IndexSlice
dfs_episode1

tuple                                          action                           
measurement                                        50                           
timestep                   2023-06-12 17:03:03.683607                           
extra                                             0.0  0.25   0.5  0.75   1.0   
timestamp                                                                       
2023-06-12 18:03:03.583607                       13.0  14.0  15.0  16.0  17.0  \
2023-06-12 19:03:03.583607                       13.0  14.0  15.0  16.0  17.0   
2023-06-12 20:03:03.583607                       13.0  14.0  15.0  16.0  17.0   
2023-06-12 21:03:03.583607                       13.0  14.0  15.0  16.0  17.0   
2023-06-12 22:03:03.583607                       13.0  14.0  15.0  16.0  17.0   

tuple                                                                           
measurement                                        60                           
timestep                   2023-06-12 17:03:03.703607                           
extra                                             0.0  0.25   0.5  0.75   1.0   
timestamp                                                                       
2023-06-12 18:03:03.583607                       18.0  19.0  20.0  21.0  22.0  \
2023-06-12 19:03:03.583607                       18.0  19.0  20.0  21.0  22.0   
2023-06-12 20:03:03.583607                       18.0  19.0  20.0  21.0  22.0   
2023-06-12 21:03:03.583607                       18.0  19.0  20.0  21.0  22.0   
2023-06-12 22:03:03.583607                       18.0  19.0  20.0  21.0  22.0   

tuple                       ...                      state   
measurement                 ...                      brake   
timestep                    ... 2023-06-12 17:03:03.623607   
extra                       ...                              
timestamp                   ...                              
2023-06-12 18:03:03.583607  ...                         10  \
2023-06-12 19:03:03.583607  ...                         10   
2023-06-12 20:03:03.583607  ...                         10   
2023-06-12 21:03:03.583607  ...                         10   
2023-06-12 22:03:03.583607  ...                         10   

tuple                                                   
measurement                                             
timestep                   2023-06-12 17:03:03.643607   
extra                                                   
timestamp                                               
2023-06-12 18:03:03.583607                         11  \
2023-06-12 19:03:03.583607                         11   
2023-06-12 20:03:03.583607                         11   
2023-06-12 21:03:03.583607                         11   
2023-06-12 22:03:03.583607                         11   

tuple                                                   
measurement                                    thrust   
timestep                   2023-06-12 17:03:03.583607   
extra                                                   
timestamp                                               
2023-06-12 18:03:03.583607                          4  \
2023-06-12 19:03:03.583607                          4   
2023-06-12 20:03:03.583607                          4   
2023-06-12 21:03:03.583607                          4   
2023-06-12 22:03:03.583607                          4   

tuple                                                   
measurement                                             
timestep                   2023-06-12 17:03:03.603607   
extra                                                   
timestamp                                               
2023-06-12 18:03:03.583607                          5  \
2023-06-12 19:03:03.583607                          5   
2023-06-12 20:03:03.583607                          5   
2023-06-12 21:03:03.583607                          5   
2023-06-12 22:03:03.583607                          5   

tuple                                       

In [43]:
dfs_episode1.loc[:, idx['action':'state', :, :, :]] = dfs_episode1.loc[
    :, idx['action':'state', :, :, :]
].astype('int')
dfs_episode1
dfs_episode1.dtypes
dfs_episode1.columns

tuple                                          action                     
measurement                                        50                     
timestep                   2023-06-12 17:03:03.683607                     
extra                                             0.0 0.25 0.5 0.75 1.0   
timestamp                                                                 
2023-06-12 18:03:03.583607                         13   14  15   16  17  \
2023-06-12 19:03:03.583607                         13   14  15   16  17   
2023-06-12 20:03:03.583607                         13   14  15   16  17   
2023-06-12 21:03:03.583607                         13   14  15   16  17   
2023-06-12 22:03:03.583607                         13   14  15   16  17   

tuple                                                                    ...   
measurement                                        60                    ...   
timestep                   2023-06-12 17:03:03.703607                    ...   
extra                                             0.0 0.25 0.5 0.75 1.0  ...   
timestamp                                                                ...   
2023-06-12 18:03:03.583607                         18   19  20   21  22  ...  \
2023-06-12 19:03:03.583607                         18   19  20   21  22  ...   
2023-06-12 20:03:03.583607                         18   19  20   21  22  ...   
2023-06-12 21:03:03.583607                         18   19  20   21  22  ...   
2023-06-12 22:03:03.583607                         18   19  20   21  22  ...   

tuple                                           state   
measurement                                     brake   
timestep                   2023-06-12 17:03:03.623607   
extra                                                   
timestamp                                               
2023-06-12 18:03:03.583607                         10  \
2023-06-12 19:03:03.583607                         10   
2023-06-12 20:03:03.583607                         10   
2023-06-12 21:03:03.583607                         10   
2023-06-12 22:03:03.583607                         10   

tuple                                                   
measurement                                             
timestep                   2023-06-12 17:03:03.643607   
extra                                                   
timestamp                                               
2023-06-12 18:03:03.583607                         11  \
2023-06-12 19:03:03.583607                         11   
2023-06-12 20:03:03.583607                         11   
2023-06-12 21:03:03.583607                         11   
2023-06-12 22:03:03.583607                         11   

tuple                                                   
measurement                                    thrust   
timestep                   2023-06-12 17:03:03.583607   
extra                                                   
timestamp                                               
2023-06-12 18:03:03.583607                          4  \
2023-06-12 19:03:03.583607                          4   
2023-06-12 20:03:03.583607                          4   
2023-06-12 21:03:03.583607                          4   
2023-06-12 22:03:03.583607                          4   

tuple                                                   
measurement                                             
timestep                   2023-06-12 17:03:03.603607   
extra                                                   
timestamp                                               
2023-06-12 18:03:03.583607                          5  \
2023-06-12 19:03:03.583607                          5   
2023-06-12 20:03:03.583607                          5   
2023-06-12 21:03:03.583607                          5   
2023-06-12 22:03:03.583607                          5   

tuple                                                   
measurement                                             
timestep                   2023-06-12 17:03:03.623

tuple   measurement  timestep                    extra
action  50           2023-06-12 17:03:03.683607  0.0      object
                                                 0.25     object
                                                 0.5      object
                                                 0.75     object
                                                 1.0      object
        60           2023-06-12 17:03:03.703607  0.0      object
                                                 0.25     object
                                                 0.5      object
                                                 0.75     object
                                                 1.0      object
        70           2023-06-12 17:03:03.723607  0.0      object
                                                 0.25     object
                                                 0.5      object
                                                 0.75     object
                                   

MultiIndex([('action',         50, 2023-06-12 17:03:03.683607,  0.0),
            ('action',         50, 2023-06-12 17:03:03.683607, 0.25),
            ('action',         50, 2023-06-12 17:03:03.683607,  0.5),
            ('action',         50, 2023-06-12 17:03:03.683607, 0.75),
            ('action',         50, 2023-06-12 17:03:03.683607,  1.0),
            ('action',         60, 2023-06-12 17:03:03.703607,  0.0),
            ('action',         60, 2023-06-12 17:03:03.703607, 0.25),
            ('action',         60, 2023-06-12 17:03:03.703607,  0.5),
            ('action',         60, 2023-06-12 17:03:03.703607, 0.75),
            ('action',         60, 2023-06-12 17:03:03.703607,  1.0),
            ('action',         70, 2023-06-12 17:03:03.723607,  0.0),
            ('action',         70, 2023-06-12 17:03:03.723607, 0.25),
            ('action',         70, 2023-06-12 17:03:03.723607,  0.5),
            ('action',         70, 2023-06-12 17:03:03.723607, 0.75),
            ('action

In [44]:
dfs_episode = dfs_epi.copy()
dfs_episode.set_index(('timestamp', '', 0, ''), inplace=True)
dfs_episode.index.name = 'timestamp'
idx = pd.IndexSlice

# dfs_episode = dfs_episode.astype({idx['state', 'velocity', :]: 'int'})  # Error unhashable type: 'slice' for dictionary
# sorted_idx = **(idx['state', :, :]:'int')
# sorted_idx
dfs_episode['state'] = dfs_episode['state'].astype(
    'float'
)  # float16 not allowed in parquet
dfs_episode['action'] = dfs_episode['action'].astype('float')
dfs_episode['reward'] = dfs_episode['reward'].astype('float')
dfs_episode['nstate'] = dfs_episode['nstate'].astype('float')
dfs_episode
dfs_episode.dtypes
# dfs_episode.columns
# dfs_epi
# dfs_episode


tuple                                           state   
measurement                                     brake   
timestep                   2023-06-12 17:03:03.583607   
extra                                                   
timestamp                                               
2023-06-12 18:03:03.583607                        8.0  \
2023-06-12 19:03:03.583607                        8.0   
2023-06-12 20:03:03.583607                        8.0   
2023-06-12 21:03:03.583607                        8.0   
2023-06-12 22:03:03.583607                        8.0   

tuple                                                   
measurement                                             
timestep                   2023-06-12 17:03:03.603607   
extra                                                   
timestamp                                               
2023-06-12 18:03:03.583607                        9.0  \
2023-06-12 19:03:03.583607                        9.0   
2023-06-12 20:03:03.583607                        9.0   
2023-06-12 21:03:03.583607                        9.0   
2023-06-12 22:03:03.583607                        9.0   

tuple                                                   
measurement                                             
timestep                   2023-06-12 17:03:03.623607   
extra                                                   
timestamp                                               
2023-06-12 18:03:03.583607                       10.0  \
2023-06-12 19:03:03.583607                       10.0   
2023-06-12 20:03:03.583607                       10.0   
2023-06-12 21:03:03.583607                       10.0   
2023-06-12 22:03:03.583607                       10.0   

tuple                                                   
measurement                                             
timestep                   2023-06-12 17:03:03.643607   
extra                                                   
timestamp                                               
2023-06-12 18:03:03.583607                       11.0  \
2023-06-12 19:03:03.583607                       11.0   
2023-06-12 20:03:03.583607                       11.0   
2023-06-12 21:03:03.583607                       11.0   
2023-06-12 22:03:03.583607                       11.0   

tuple                                                   
measurement                                    thrust   
timestep                   2023-06-12 17:03:03.583607   
extra                                                   
timestamp                                               
2023-06-12 18:03:03.583607                        4.0  \
2023-06-12 19:03:03.583607                        4.0   
2023-06-12 20:03:03.583607                        4.0   
2023-06-12 21:03:03.583607                        4.0   
2023-06-12 22:03:03.583607                        4.0   

tuple                                                   
measurement                                             
timestep                   2023-06-12 17:03:03.603607   
extra                                                   
timestamp                                               
2023-06-12 18:03:03.583607                        5.0  \
2023-06-12 19:03:03.583607                        5.0   
2023-06-12 20:03:03.583607                        5.0   
2023-06-12 21:03:03.583607                        5.0   
2023-06-12 22:03:03.583607                        5.0   

tuple                                                   
measurement                                             
timestep                   2023-06-12 17:03:03.623607   
extra                                                   
timestamp                                               
2023-06-12 18:03:03.583607                        6.0  \
2023-06-12 19:03:03.583607                        6.0   
2023-06-12 20:03:03.583607                        6.0   
2023-06-12 21:03:03.583607                        6.0   
2023-06-12 22:03:03.583607                        6.0   

tup

tuple   measurement  timestep                    extra
state   brake        2023-06-12 17:03:03.583607           float64
                     2023-06-12 17:03:03.603607           float64
                     2023-06-12 17:03:03.623607           float64
                     2023-06-12 17:03:03.643607           float64
        thrust       2023-06-12 17:03:03.583607           float64
                     2023-06-12 17:03:03.603607           float64
                     2023-06-12 17:03:03.623607           float64
                     2023-06-12 17:03:03.643607           float64
        velocity     2023-06-12 17:03:03.583607           float64
                     2023-06-12 17:03:03.603607           float64
                     2023-06-12 17:03:03.623607           float64
                     2023-06-12 17:03:03.643607           float64
reward               0                                    float64
action  50           2023-06-12 17:03:03.683607  0.0      float64
                     

## prepend two levels of index "vehicle" and "driver" to the DataFrame object

In [45]:
dfs_episode = pd.concat(
    [dfs_episode], keys=[drivers_by_id['wang-cheng'].pid], names=['driver']
)
dfs_episode = pd.concat(
    [dfs_episode], keys=[trucks_by_name['VB7'].vid], names=['vehicle']
)

In [46]:

dfs_episode.columns.set_names(
    ['tuple', 'measurement', 'timestep', 'extra'], level=[0, 1, 2, 3], inplace=True
)
dfs_episode
dfs_episode.dtypes


tuple                                                              state   
measurement                                                        brake   
timestep                                      2023-06-12 17:03:03.583607   
extra                                                                      
vehicle driver     timestamp                                               
VB7     wang-cheng 2023-06-12 18:03:03.583607                        8.0  \
                   2023-06-12 19:03:03.583607                        8.0   
                   2023-06-12 20:03:03.583607                        8.0   
                   2023-06-12 21:03:03.583607                        8.0   
                   2023-06-12 22:03:03.583607                        8.0   

tuple                                                                      
measurement                                                                
timestep                                      2023-06-12 17:03:03.603607   
extra                                                                      
vehicle driver     timestamp                                               
VB7     wang-cheng 2023-06-12 18:03:03.583607                        9.0  \
                   2023-06-12 19:03:03.583607                        9.0   
                   2023-06-12 20:03:03.583607                        9.0   
                   2023-06-12 21:03:03.583607                        9.0   
                   2023-06-12 22:03:03.583607                        9.0   

tuple                                                                      
measurement                                                                
timestep                                      2023-06-12 17:03:03.623607   
extra                                                                      
vehicle driver     timestamp                                               
VB7     wang-cheng 2023-06-12 18:03:03.583607                       10.0  \
                   2023-06-12 19:03:03.583607                       10.0   
                   2023-06-12 20:03:03.583607                       10.0   
                   2023-06-12 21:03:03.583607                       10.0   
                   2023-06-12 22:03:03.583607                       10.0   

tuple                                                                      
measurement                                                                
timestep                                      2023-06-12 17:03:03.643607   
extra                                                                      
vehicle driver     timestamp                                               
VB7     wang-cheng 2023-06-12 18:03:03.583607                       11.0  \
                   2023-06-12 19:03:03.583607                       11.0   
                   2023-06-12 20:03:03.583607                       11.0   
                   2023-06-12 21:03:03.583607                       11.0   
                   2023-06-12 22:03:03.583607                       11.0   

tuple                                                                      
measurement                                                       thrust   
timestep                                      2023-06-12 17:03:03.583607   
extra                                                                      
vehicle driver     timestamp                                               
VB7     wang-cheng 2023-06-12 18:03:03.583607                        4.0  \
                   2023-06-12 19:03:03.583607                        4.0   
                   2023-06-12 20:03:03.583607                        4.0   
                   2023-06-12 21:03:03.583607                        4.0   
                   2023-06-12 22:03:03.583607                        4.0   

tuple                                                                      
measurement                                                                
timestep                                   

tuple   measurement  timestep                    extra
state   brake        2023-06-12 17:03:03.583607           float64
                     2023-06-12 17:03:03.603607           float64
                     2023-06-12 17:03:03.623607           float64
                     2023-06-12 17:03:03.643607           float64
        thrust       2023-06-12 17:03:03.583607           float64
                     2023-06-12 17:03:03.603607           float64
                     2023-06-12 17:03:03.623607           float64
                     2023-06-12 17:03:03.643607           float64
        velocity     2023-06-12 17:03:03.583607           float64
                     2023-06-12 17:03:03.603607           float64
                     2023-06-12 17:03:03.623607           float64
                     2023-06-12 17:03:03.643607           float64
reward               0                                    float64
action  50           2023-06-12 17:03:03.683607  0.0      float64
                     

### Convert columns types to int and float16

In [27]:
# idx = pd.IndexSlice
# dfs_episode.loc[:, idx['state', :, :]] = dfs_episode.loc[:, idx['state', :, :]].astype(
#     'int'
# )
# dfs_episode.loc[:, idx['action', :, :]] = dfs_episode.loc[
#     :, idx['action', :, :]
# ].astype('float16')
# dfs_episode.loc[:, idx['reward', :, :]] = dfs_episode.loc[
#     :, idx['reward', :, :]
# ].astype('float16')
# dfs_episode.loc[:, idx['nstate', :, :]] = dfs_episode.loc[
#     :, idx['nstate', :, :]
# ].astype('float16')
#
# vel_1 = dfs_episode[[('state', 'velocity', 1)]]
# vel_1.dtypes
# vel_1.index
# vel_1.values
# # type(vel_1)
# vel_1.iloc[0]
# type(vel_1.iloc[0])
# type(vel_1.iloc[0].values[0])

## Add a level of index for episode

In [47]:
episodestart = ts - pd.Timedelta(1, 'h')
dfs_episode = pd.concat([dfs_episode], keys=[ts], names=['episodestart'])
dfs_episode.sort_index(inplace=True)
dfs_episode.index
dfs_episode.columns
dfs_episode

MultiIndex([('2023-06-12 17:03:03.583607', 'VB7', 'wang-cheng', ...),
            ('2023-06-12 17:03:03.583607', 'VB7', 'wang-cheng', ...),
            ('2023-06-12 17:03:03.583607', 'VB7', 'wang-cheng', ...),
            ('2023-06-12 17:03:03.583607', 'VB7', 'wang-cheng', ...),
            ('2023-06-12 17:03:03.583607', 'VB7', 'wang-cheng', ...)],
           names=['episodestart', 'vehicle', 'driver', 'timestamp'])

MultiIndex([( 'state',    'brake', 2023-06-12 17:03:03.583607,   ''),
            ( 'state',    'brake', 2023-06-12 17:03:03.603607,   ''),
            ( 'state',    'brake', 2023-06-12 17:03:03.623607,   ''),
            ( 'state',    'brake', 2023-06-12 17:03:03.643607,   ''),
            ( 'state',   'thrust', 2023-06-12 17:03:03.583607,   ''),
            ( 'state',   'thrust', 2023-06-12 17:03:03.603607,   ''),
            ( 'state',   'thrust', 2023-06-12 17:03:03.623607,   ''),
            ( 'state',   'thrust', 2023-06-12 17:03:03.643607,   ''),
            ( 'state', 'velocity', 2023-06-12 17:03:03.583607,   ''),
            ( 'state', 'velocity', 2023-06-12 17:03:03.603607,   ''),
            ( 'state', 'velocity', 2023-06-12 17:03:03.623607,   ''),
            ( 'state', 'velocity', 2023-06-12 17:03:03.643607,   ''),
            ('reward',         '',                          0,   ''),
            ('action',         50, 2023-06-12 17:03:03.683607,  0.0),
            ('action

tuple                                                                                         state   
measurement                                                                                   brake   
timestep                                                                 2023-06-12 17:03:03.583607   
extra                                                                                                 
episodestart               vehicle driver     timestamp                                               
2023-06-12 17:03:03.583607 VB7     wang-cheng 2023-06-12 18:03:03.583607                        8.0  \
                                              2023-06-12 19:03:03.583607                        8.0   
                                              2023-06-12 20:03:03.583607                        8.0   
                                              2023-06-12 21:03:03.583607                        8.0   
                                              2023-06-12 22:03:03.583607                        8.0   

tuple                                                                                                 
measurement                                                                                           
timestep                                                                 2023-06-12 17:03:03.603607   
extra                                                                                                 
episodestart               vehicle driver     timestamp                                               
2023-06-12 17:03:03.583607 VB7     wang-cheng 2023-06-12 18:03:03.583607                        9.0  \
                                              2023-06-12 19:03:03.583607                        9.0   
                                              2023-06-12 20:03:03.583607                        9.0   
                                              2023-06-12 21:03:03.583607                        9.0   
                                              2023-06-12 22:03:03.583607                        9.0   

tuple                                                                                                 
measurement                                                                                           
timestep                                                                 2023-06-12 17:03:03.623607   
extra                                                                                                 
episodestart               vehicle driver     timestamp                                               
2023-06-12 17:03:03.583607 VB7     wang-cheng 2023-06-12 18:03:03.583607                       10.0  \
                                              2023-06-12 19:03:03.583607                       10.0   
                                              2023-06-12 20:03:03.583607                       10.0   
                                              2023-06-12 21:03:03.583607                       10.0   
                                              2023-06-12 22:03:03.583607                       10.0   

tuple                                                                                                 
measurement                                                                                           
timestep                                                                 2023-06-12 17:03:03.643607   
extra                                                                                                 
episodestart               vehicle driver     timestamp                                               
2023-06-12 17:03:03.583607 VB7     wang-cheng 2023-06-12 18:03:03.583607                       11.0  \
                                              2023-06-12 19:03:03.583607                       11.0   
                                              2023-06-12 20:03:03.583607                       11.0   
                                              2023-06-12 21:03:03.583607           

In [48]:

ts_new = pd.to_datetime((datetime.now()))
ts_new
episodestart = ts_new - pd.Timedelta(2, 'd')
episodestart

dfs_episode0 = dfs_episode.copy()
dfs_episode0.index = dfs_episode0.index.set_levels([episodestart], level='episodestart')
dfs_episode0.index = dfs_episode0.index.set_levels(
    [[trucks[0].vid], [drivers[1].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode1 = dfs_episode.copy()
dfs_episode1.index = dfs_episode1.index.set_levels(
    [episodestart - pd.Timedelta(3, 'd')], level='episodestart'
)
dfs_episode1.index = dfs_episode1.index.set_levels(
    [[trucks[1].vid], [drivers[0].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode2 = dfs_episode.copy()
dfs_episode2.index = dfs_episode2.index.set_levels(
    [episodestart - pd.Timedelta(4, 'd')], level='episodestart'
)
dfs_episode2.index = dfs_episode2.index.set_levels(
    [[trucks[0].vid], [drivers[0].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode3 = dfs_episode.copy()
dfs_episode3.index = dfs_episode3.index.set_levels(
    [episodestart - pd.Timedelta(5, 'd')], level='episodestart'
)
dfs_episode3.index = dfs_episode3.index.set_levels(
    [[trucks[0].vid], [drivers[1].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode0



Timestamp('2023-06-12 17:17:48.322380')

Timestamp('2023-06-10 17:17:48.322380')

NameError: name 'trucks' is not defined

In [ ]:
# dfs_episode_concat = pd.concat([dfs_episode, dfs_episode0], axis=0)
# display('dfs_episode_concat')
# dfs_episode_concat.index
# dfs_episode_concat.columns

In [ ]:
episodes = [dfs_episode, dfs_episode0, dfs_episode1, dfs_episode2, dfs_episode3]
try:
    dfs_episode_all = reduce(
        lambda left, right,: pd.concat([left, right], axis=0), episodes
    )
except Exception as e:
    print(e)
# dfs_episode_all.sort_index(inplace=True)
dfs_episode_all = dfs_episode_all.swaplevel(1, 0, axis=0)
dfs_episode_all = dfs_episode_all.swaplevel(1, 2, axis=0)
dfs_episode_all.sort_index(inplace=True)
dfs_episode_all = dfs_episode_all[['state', 'action', 'reward', 'nstate']]
display('dfs_episode_all')
dfs_episode_all.index
dfs_episode_all.columns

## Make MultiIndex to Rows by adding levels of "Vehicle" and "Driver"

###  reset index to move vehicle and driver to columns, preprocessing for dask manipulation

In [ ]:
dfs_episodes = dfs_episode_all.reset_index(
    level=['vehicle', 'driver']
)  # unstack vehicle and driver to columns in level 0 with default '' in level 1
# # dfs_episodes.columns
# # dfs_episode0.index = dfs_episode0.index.set_levels([[trucks[0].vid],[drivers[1].pid]], level=['vehicle', 'driver'], verify_integrity=False)
# old_columns = dfs_episodes.columns
# # old_columns[0]= (old_columns[0][0],0)
# new_columns_tuples = [(old_columns[0][0],0), (old_columns[1][0],0)] + old_columns[2:].to_list()
# prepend_columns = pd.MultiIndex.from_tuples(new_columns_tuples, names=old_columns.names)
# # prepend_columns

# dfs_episodes.columns = prepend_columns
dfs_episodes
# dfs_episodes.columns
# dfs_episodes.dtypes
# dfs_episode_all

In [ ]:
dfs_episodes.dtypes

In [ ]:
dfs_episodes.columns

In [ ]:

dfs_episodes['vehicle'] = dfs_episodes['vehicle'].astype("category")
dfs_episodes['driver'] = dfs_episodes['driver'].astype("category")
# dfs_episodes.dtypes
# dfs_episodes.columns
# dfs_episodes[['vehicle','driver']].dtypes
# dfs_episodes[['vehicle','driver']]
# dfs_episodes[['vehicle','driver']].iloc[:3].dtypes
# sliced = dfs_episodes[['vehicle']].iloc[:3].stack(level=[-1,-2])
# type(sliced)
# sliced.index
# sliced.values

dfs_episodes_sorted = dfs_episodes.sort_index(
    axis=1, level=[0, 1, 2], ascending=[True, True, True]
)
dfs_episodes_sorted.sort_index(axis=0, inplace=True)
# dfs_episodes_sorted = dfs_episodes.sort_index(axis=1, level=[0,1], ascending=[True, True])
# dfs_episodes_sorted = dfs_episodes.sort_index(axis=0, level=[0,1], ascending=[True, True])
# dfs_episodes_sorted.sort_index(axis=1,inplace=True)
dfs_episodes_sorted
dfs_episodes_sorted.columns

# dfs_episodes[['driver'], ['vehicle']] = dfs_episodes[['driver', 'vehicle']].apply(lambda x: x.cat.codes)

## dataframe slicing

In [49]:
ts
ep_start = ts - pd.Timedelta(9, 'D')
ep_start
# time_slice
episode_slice = pd.date_range(ep_start, periods=6, freq='D')
episode_slice

Timestamp('2023-06-12 17:03:03.583607')

Timestamp('2023-06-03 17:03:03.583607')

DatetimeIndex(['2023-06-03 17:03:03.583607', '2023-06-04 17:03:03.583607',
               '2023-06-05 17:03:03.583607', '2023-06-06 17:03:03.583607',
               '2023-06-07 17:03:03.583607', '2023-06-08 17:03:03.583607'],
              dtype='datetime64[ns]', freq='D')

In [50]:
timestamp_slice = pd.date_range(ts, periods=3, freq='H') + pd.Timedelta(10, 'min')
timestamp_slice

DatetimeIndex(['2023-06-12 17:13:03.583607', '2023-06-12 18:13:03.583607',
               '2023-06-12 19:13:03.583607'],
              dtype='datetime64[ns]', freq='H')

In [51]:
dfs_episodes_sorted

NameError: name 'dfs_episodes_sorted' is not defined

In [ ]:
dfs_episodes_sorted.index
dfs_episodes_sorted.columns

In [ ]:
# dfs_episode_all.loc['VB7', 'zheng-longfei', episode_slice[0]:episode_slice[3]]
# dfs_selected_episodes = dfs_episodes_sorted.loc[trucks[0].vid, drivers[1].pid, episode_slice[0]:episode_slice[3], timestamp_slice[0]:timestamp_slice[2]]
idx = pd.IndexSlice
dfs_selected_episodes = dfs_episodes_sorted.loc[
    idx[episode_slice[0] : episode_slice[5], timestamp_slice[0] : timestamp_slice[2]],
    idx['action':'driver'],
]
dfs_selected_episodes
# dfs_selected_episodes[['action', 'reward']]

In [ ]:
# dfs_selected_episodes_columns = dfs_episode_all.loc['VB7', 'zheng-longfei', episode_slice[0]:episode_slice[3], timestamp_slice[0]:timestamp_slice[2]][['action', 'reward']]
# dfs_selected_episodes_columns

In [ ]:
# dfs_episode_all_sorted = dfs_episode_all.sort_index(axis=1)
# dfs_episode_all_sorted = dfs_episode_all_sorted.sort_index(axis=0)
# dfs_episode_all_sorted

In [ ]:

# dfs_selected_episodes_columns = dfs_episode_all_sorted.loc[(slice('VB6'), slice('wang-cheng'), slice(None), slice(None)),:]
# idx = pd.IndexSlice
# dfs_episode_all_sorted.loc[(idx['VB6'], idx['wang-cheng','zheng-longfei'], slice(None), slice(None)),:]
# dfs_selected_episodes_columns = dfs_episode_all_sorted.loc[(idx['VB7'], idx['zheng-longfei'], slice(None), slice(None)),['state','n_state']]
# dfs_selected_episodes_columns = dfs_episodes_sorted.loc[(idx['VB7'], idx['zheng-longfei'], slice(None), slice(None)),['state','n_state']]
# dfs_selected_episodes_columns

In [ ]:

ts_new = pd.to_datetime((datetime.now()))
episodestart = ts_new - pd.Timedelta(10, 'd') - pd.Timedelta(6, 'h')
episodestart
episode_slice = pd.date_range(episodestart, periods=20, freq='d') + pd.Timedelta(
    10, 'min'
)
episode_slice
ts
time_slice = pd.date_range(ts, periods=6, freq='h') - pd.Timedelta(10, 'min')
time_slice


In [ ]:

episode_slice[4]: episode_slice[8]
idx[episode_slice[4] : episode_slice[8]]
idx[episode_slice[4] : episode_slice[8], slice(None)]

In [ ]:
episode_slice[4]
episode_slice[11]
# dfs_selected_episodes = dfs_episodes_sorted.loc[idx['VB7'], slice(None), episode_slice[4]:episode_slice[11]]
# dfs_selected_episodes
# dfs_selected_episodes = dfs_episodes_sorted.loc[slice(None), idx['zheng-longfei'], episode_slice[4]:episode_slice[11]]
# dfs_selected_episodes
# dfs_selected_episodes = dfs_episode_all.loc[slice(None), idx['zheng-longfei'], episode_slice[4]:episode_slice[11], time_slice[0]:time_slice[3]]
# dfs_selected_episodes
# epislice = idx(episode_slice[4]:episode_slice[11])

# dfs_selected_episodes = dfs_episodes_sorted.loc[idx[episode_slice[4]:episode_slice[11],slice(None)], idx[dfs_episodes_sorted['driver']==drivers[0].pid]]
dfs_selected_episodes1 = dfs_episodes_sorted.loc[
    dfs_episodes_sorted['driver'] == drivers[0].pid
]
dfs_selected_episodes1
dfs_selected_episodes2 = dfs_episodes_sorted.loc[
    dfs_episodes_sorted['driver'] == drivers[0].pid
].loc[idx[episode_slice[4] : episode_slice[8]]]
dfs_selected_episodes2
dfs_selected_episodes3 = dfs_episodes_sorted.loc[
    (dfs_episodes_sorted['driver'] == drivers[0].pid)
    & (dfs_episodes_sorted['vehicle'] == trucks[0].vid)
].loc[idx[episode_slice[4] : episode_slice[8]]]
dfs_selected_episodes3
# dfs_selected_episodes31 = dfs_episodes_sorted.loc[idx[episode_slice[4]:episode_slice[8],slice(None)]].loc[ (dfs_episodes_sorted['driver']==drivers[0].pid) & (dfs_episodes_sorted['vehicle']==trucks[0].vid)]
# dfs_selected_episodes31 = dfs_episodes_sorted.loc[ (dfs_episodes_sorted['driver']==drivers[0].pid) & (dfs_episodes_sorted['vehicle']==trucks[0].vid) & (idx[episode_slice[4]:episode_slice[8],slice(None)])]
# dfs_selected_episodes31
# dfs_selected_episodes3.index
# dfs_selected_episodes3.columns
# dfs_selected_episodes = dfs_episodes_sorted.loc[slice(None), idx['zheng-longfei'], episode_slice[4]:episode_slice[11]]
# dfs_selected_episodes
# dfs_selected_episodes = dfs_episode_all.loc[slice(None), idx['zheng-longfei'], episode_slice[4]:episode_slice[11], time_slice[0]:time_slice[3]]
# dfs_selected_episodes


In [ ]:
idx[time_slice[0] : time_slice[3]]

In [ ]:

dfs_selected_episodes3.loc[:, idx[:, :, 0:2]]
# dfs_selected_episodes3.loc[idx[:,time_slice[0]:time_slice[2]],:]
dfs_selected_episodes3.loc[idx[:, time_slice[0] : time_slice[3]], :]

In [ ]:
dfs_selected_episodes3.index
# dfs_selected_episodes3.columns
time_slice[0]
time_slice[3]
idx[:, time_slice[0] : time_slice[3]]
dfs_selected_episodes4 = dfs_selected_episodes3.loc[
    idx[:, time_slice[0] : time_slice[3]], :
]
# dfs_selected_episodes4 = dfs_selected_episodes3.loc[idx[:,time_slice[0]:time_slice[3],:],:]

dfs_selected_episodes4
dfs_selected_episodes41 = dfs_selected_episodes3.loc[
    idx[:, time_slice[0] : time_slice[3]], 'action':'driver'
]
dfs_selected_episodes41
# dfs_selected_episodes4 = dfs_episodes_sorted.loc[ (dfs_episodes_sorted['driver']==drivers[0].pid) & (dfs_episodes_sorted['vehicle']==trucks[0].vid)].loc[idx[episode_slice[4]:episode_slice[8],time_slice[0]:time_slice[3]]]
# dfs_selected_episodes4

In [ ]:

# dfs_selected_episodes = dfs_episode_all.loc['VB7', 'zheng-longfei', episode_slice[0]:episode_slice[3], timestamp_slice[0]:timestamp_slice[2]]
# dfs_selected_episodes_columns = dfs_episode_all_sorted.loc[(idx['VB7'], idx['zheng-longfei'], slice(None), slice(None)),['state','n_state']]
# dfs_selected_episodes_columns


In [ ]:
dfs_episodes_sorted
# dfs_episode.loc[:,'state']
dfs_slice = dfs_episodes_sorted.loc[
    dfs_episodes_sorted['vehicle'] == 'MP73', ['vehicle', 'driver', 'state', 'reward']
]
dfs_slice

# Saved episodes as parquet in multi-level folders "vehicle/driver/episode.parquet"

In [ ]:
import os
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import numpy as np
import pandas as pd
from datetime import datetime

import pyarrow as pa
import pyarrow.parquet as pq
import platform

print('Python: ', platform.python_version())
print('pandas: ', pd.__version__)
print('pyarrow: ', pa.__version__)


In [ ]:
os.chdir('../../data')

In [ ]:

os.getcwd()

## create parquet files

In [ ]:
dfs_episodes_sorted
dfs_episodes_sorted.index
dfs_episodes_sorted.columns
dfs_episodes_sorted.dtypes

In [ ]:
# dfs_episode4dask = dfs_episode_all_sorted.swaplevel(0,-1, axis=0).swaplevel(1,3, axis=0).swaplevel(2,3, axis=0).unstack()
# dfs_episode4dask = dfs_episode4dask.swaplevel(0,-1, axis=1).swaplevel(-2,-1, axis=1)
# dfs_episode4dask
# # dfs_episode4dask = dfs_episode4dask.swaplevel(0,-1, axis=1).swaplevel(-2,-1, axis=1)
# # dfs_episode4dask.index
# # dfs_episode4dask.index
# # dfs_episode4dask.columns
# # dfs_episode4dask
# # dfs_episode_all_stack = dfs_episode_all.unstack(level=[-1,-2,-3,])
# # dfs_episode_all_stack

In [ ]:
# dfs_episode4dask1 = dfs_episode4dask.unstack().swaplevel(-2,-1, axis=1).swaplevel(-2,-3,axis=1).swaplevel(-3,-4,axis=1)
# # dfs_episode4dask1
# dfs_episode4dask2 = dfs_episode4dask1.unstack().swaplevel(-2,-1, axis=1).swaplevel(-2,-3,axis=1).swaplevel(-3,-4,axis=1).swaplevel(-4,-5,axis=1)
# dfs_episode4dask2.sort_index(axis=1, inplace=True)
# dfs_episode4dask2


### decoding multiindex dataframe to flat index for Dask datframe

In [ ]:
dfs_episodes_sorted.columns
cols = dfs_episodes_sorted.columns.to_flat_index()
cols

cols_str = [
    f'{x[0]}_{x[1]}_{x[2]}'
    if (x[1] != '')
    else f'{x[0]}__{x[2]}'
    if (x[2] != '')
    else f'{x[0]}'
    for x in cols
]
cols_str

In [ ]:
dfs_episodes_sorted_flat_col = dfs_episodes_sorted.copy()
cols = dfs_episodes_sorted.columns.to_flat_index()
dfs_episodes_sorted_flat_col.columns = [
    f'{x[0]}_{x[1]}_{x[2]}'
    if (x[1] != '')
    else f'{x[0]}__{x[2]}'  # !!! Very Important!!! dunder means the next of the next level! but str.splitt('_'), you get the '' in between for free!
    if (x[2] != '')
    else f'{x[0]}'
    for x in cols
]
dfs_episodes_sorted_flat_ind = dfs_episodes_sorted.copy()
idx = dfs_episodes_sorted.index.to_flat_index()
dfs_episodes_sorted_flat_ind.index = [
    f'{x[0]}_{x[1]}' if x[1] != '' else f'x{0}' for x in idx
]
# dfs_episodes_sorted_flat_ind
dfs_episodes_sorted_flat = dfs_episodes_sorted.copy()
dfs_episodes_sorted_flat.columns = [
    f'{x[0]}_{x[1]}_{x[2]}'
    if (x[1] != '')
    else f'{x[0]}__{x[2]}'
    if (x[2] != '')
    else f'{x[0]}'
    for x in cols
]
dfs_episodes_sorted_flat.index = [
    f'{idx[0]}_{idx[1]}' if idx[1] != '' else f'{idx[0]}'
    for idx in dfs_episodes_sorted.index.to_flat_index()
]
dfs_episodes_sorted_flat



In [ ]:
dfs_episodes_sorted_flat_ind
dfs_episodes_sorted_flat_col
dfs_episodes_sorted_flat
dfs_episodes_sorted_flat.dtypes

In [ ]:
dfs_episodes_sorted_flat['action_row0_0'].iloc[0:2].values.dtype

In [ ]:
try:
    ddf_episodes = dd.from_pandas(dfs_episodes_sorted_flat, npartitions=1)
except Exception as e:
    print(e)

try:
    ddf_episodes.index
    ddf_episodes.columns
except Exception as e:
    print(e)

ddf_episodes

In [ ]:

custom_metadata = {
    b'eos': b'{"length":[3, 600, 5],"timezeone":"sh","units":["kph","pct","pct"],"data_propensity":[50, 1, 4],"Dataset Description": "MP vehicle from TBox"}'
}
custom_metadata

In [ ]:
try:
    with ProgressBar():
        ddf_episodes.to_parquet(
            'eos_episodes',
            engine='pyarrow',
            compression='snappy',
            partition_on=['vehicle', 'driver'],
            write_metadata_file=True,
            custom_metadata=custom_metadata,
        )
except Exception as e:
    print(e)

### Read saved parquet file

In [52]:
ddf = dd.read_parquet(
    'eos_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
ddf.head(5)
ddf.dtypes
ddf.npartitions
ddf.divisions
ddf._meta

NameError: name 'dd' is not defined

### Get customized metadata with pyarrow

In [ ]:

import pyarrow as pa
import pyarrow.parquet as pq
import json

# print('pyarrow: ', pa.__version__)

# table = pq.read_table('ddf_episodes')
# table_meta = table.schema.metadata[b'eos']
# print(table_meta)
# # custom_metadata_info = json.loads(table.schema.metadata[b'eos'])
# # print(custom_metadata_info)
# try:
#     custom_meta_info = json.loads(table_meta)
# except Exception as e:
#     print(f'Exception: {e}')

# print(custom_meta_info)

# # Print formatted output of the dichtionary custom_meta_info:
# for key, val in custom_meta_info.items():
#     print('{:15}: {}'.format(key, val))

# ALTERNATE:

table_meta = pq.read_metadata('eos_episodes/_common_metadata')
# print(table_meta)
# print(table_meta.metadata[b'eos'])
custom_meta_info = table_meta.metadata[b'eos']
custom_meta_info = json.loads(custom_meta_info)
# custom_meta_info

for key, val in custom_meta_info.items():
    print(f'{key}: {val}')

# table_meta = pq.read_metadata('ddf_episodes/_metadata')
# # print(table_meta)
# # print(table_meta.metadata[b'eos'])
# custom_meta_info = table_meta.metadata[b'eos']
# custom_meta_info = json.loads(custom_meta_info)
# custom_meta_info

### encoding flat_index of dask dataframe to multiindex

In [53]:
df_episodes_read = ddf.compute()
df_episodes_read

NameError: name 'ddf' is not defined

In [ ]:
multi_cols = [tuple(col.split('_')) for col in df_episodes_read.columns]
multi_cols
# multi_cols = [(col[0], int(col[1])) if len(col)==2 else col for col in multi_cols]
# multi_cols
df_episodes_read_multicol = df_episodes_read.copy()
multi_idx = pd.MultiIndex.from_tuples(multi_cols)
# multi_idx
multi_idx.names = ['tuple', 'measurement', 'step']
driver_vehicle = [idx for idx in multi_idx if idx[0] == 'driver' or idx[0] == 'vehicle']
driver_vehicle
# multi_idx = multi_idx.set_levels([multi_idx.levels[0], multi_idx.levels[1].astype(int)])
# multi_idx

In [ ]:
# pd.Series([1, np.nan,2,None])
# pd.Series([1,np.nan,2,None,pd.NA])

In [ ]:
multi_tpl = [tuple(col.split('_')) for col in df_episodes_read.columns]
# multi_tpl
# multi_cols = [(col[0], int(col[1])) if len(col)==2 else col for col in multi_cols]
# multi_cols
df_episodes_read_multicol = df_episodes_read.copy()
multi_col = pd.MultiIndex.from_tuples(multi_tpl)
# multi_idx
# multi_col
i1 = multi_col.get_level_values(0)
i1 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i1
]  # convert index of level 2 type to int and '' if NA
i1
# i2 = multi_col.get_level_values(1).fillna('') # must be null string instead of the default pd.NA or np.nan
# i2 = [idx if isinstance(idx, int) else '' for idx in i2]
i2 = multi_col.get_level_values(
    1
)  # must be null string instead of the default pd.NA or np.nan
i2 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i2
]  # convert index of level 2 type to int and '' if NA
i2
# i2 = [idx if isinstance(idx, str) else '' for idx in i2]
# i2.astype('int')
i3 = multi_col.get_level_values(
    2
)  # must be null string instead of the default pd.NA or np.nan
i3 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else int(idx) for idx in i3
]  # convert index of level 2 type to int and '' if NA
i3

multi_col = pd.MultiIndex.from_arrays([i1, i2, i3])
multi_col.names = ['tuple', 'measurement', 'step']
multi_col


# i2[-1]
# i2.dropna()
# i2
# l2 = ['' if i==nan else i  for i in i2]
# l2
# i2= i2[:-2].astype('int')
# i2 = i2.append(pd.Index(['','']))
# l2 = [''] + list(l2.values[:-1])
# print(l2)
# arrays = [ for i2 in l2 for i1 in l1]
# l2 = l2[:-2].astype('int') + ['', '']
# multi_col.set_levels(ll, level=['tuple','step'])
# driver_vehicle = [idx for idx in multi_idx if idx[0]=='driver' or idx[0]=='vehicle']
# multi_tpl =  [(col[0],)  if col[0]=='driver' or col[0]=='vehicle' else (col[0],int(col[1])) for col in multi_col]
# multi_idx1 = pd.MultiIndex.from_tuples(multi_idx1)
# multi_idx = multi_idx.set_levels([multi_idx.levels[0], multi_idx.levels[1].astype(int)])
# multi_col = pd.MultiIndex.from_tuples(multi_tpl)
# multi_col

In [ ]:
multi_idx = [tuple(idx.split('_')) for idx in df_episodes_read.index]
# multi_idx
df_episodes_read_multiidx = df_episodes_read.copy()
multi_idx = pd.MultiIndex.from_tuples(multi_idx)
multi_idx.names = ['episodestart', 'timestamp']
multi_idx
# len(multi_idx)

In [ ]:
df_episodes_read_multi = df_episodes_read.copy()
df_episodes_read_multi.columns = multi_col
df_episodes_read_multi.index = multi_idx
df_episodes_read_multi
df_episodes_read_multi.columns
df_episodes_read_multi.index
df_episodes_read_multi.dtypes

### recover deep multiindex from columns ['vehicle','driver']

In [ ]:
df_episodes_read_multi_setindex = df_episodes_read_multi.set_index(
    ['vehicle', 'driver', df_episodes_read_multi.index]
)
df_episodes_read_multi_setindex.index
df_episodes_read_multi_setindex.columns
df_episodes_read_multi_setindex

## append new episode to the specific parquet file

In [54]:
dfs_episode4 = dfs_episode.copy()
dfs_episode4.index = dfs_episode3.index.set_levels(
    [episodestart - pd.Timedelta(8, 'd')], level='episodestart'
)
dfs_episode4.index = dfs_episode3.index.set_levels(
    [[trucks[1].vid], [drivers[1].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode4 = dfs_episode4.swaplevel(0, 1, axis=0).swaplevel(1, 2, axis=0)
dfs_episode4.columns

# dfs_episode5 = dfs_episode.copy()
# dfs_episode5.index = dfs_episode3.index.set_levels([episodestart-pd.Timedelta(9,'d')], level='episode')
df_episodes_read_multi_setindex.columns

NameError: name 'dfs_episode3' is not defined

In [ ]:
dfs_episodes_new = pd.concat([df_episodes_read_multi_setindex, dfs_episode4], axis=0)
dfs_episodes_new
# dfs_episodes_new.columns
# dfs_episodes_new.index

#### save new dataframe back to parquet

In [ ]:

os.getcwd()

In [ ]:
dfs_episodes_new_flat = dfs_episodes_new.copy()
dfs_episodes_new_flat = dfs_episodes_new_flat.reset_index(level=['vehicle', 'driver'])
dfs_episodes_new_flat.columns

In [ ]:
dfs_episodes_new_flat.index = [
    f'{idx[0]}_{idx[1]}' if idx[1] != '' else f'{idx[0]}'
    for idx in dfs_episodes_new_flat.index.to_flat_index()
]
dfs_episodes_new_flat.columns = [
    f'{x[0]}_{x[1]}_{x[2]}'
    if (x[1] != '')
    else f'{x[0]}__{x[2]}'  # !!! dunder!!!
    if (x[2] != '')
    else f'{x[0]}'
    for x in dfs_episodes_new_flat.columns.to_flat_index()
]
# dfs_episodes_new_flat['vehicle'] = dfs_episodes_new_flat['vehicle'].astype("category")
# dfs_episodes_new_flat['driver'] = dfs_episodes_new_flat['driver'].astype("category")

dfs_episodes_new_flat.index.name = 'episodestart_timestamp'
dfs_episodes_new_flat.index
dfs_episodes_new_flat.dtypes
dfs_episodes_new_flat.columns
dfs_episodes_new_flat


In [ ]:
try:
    ddf_episodes_new = dd.from_pandas(dfs_episodes_new_flat, npartitions=1)
except Exception as e:
    print(e)

# dfs_episodes_new_flat


In [ ]:
os.getcwd()
custom_metadata = {
    b'eos': b'{"length":[6, 600, 5],"timezeone":"sh","units":["kph","pct","pct"],"data_propensity":[50, 1, 4],"Dataset Description": "MP vehicle from TBox"}'
}

try:
    with ProgressBar():
        ddf_episodes_new.to_parquet(
            'eos_episodes',
            engine='pyarrow',
            compression='snappy',
            partition_on=['vehicle', 'driver'],
            write_metadata_file=True,
            custom_metadata=custom_metadata,
        )
except Exception as e:
    print(e)

In [ ]:
ddf = dd.read_parquet(
    'eos_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
pq_meta = pq.read_metadata('eos_episodes/_common_metadata')
custom_meta_info = pq_meta.metadata[b'eos']
custom_meta_info = json.loads(custom_meta_info)
for key, val in custom_meta_info.items():
    print(f'{key}: {val}')

df = ddf.compute()

multi_tpl = [tuple(col.split('_')) for col in df.columns]
multi_col = pd.MultiIndex.from_tuples(multi_tpl)
i1 = multi_col.get_level_values(0)
i2 = multi_col.get_level_values(
    1
)  # must be null string instead of the default pd.NA or np.nan
i2 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i2
]  # convert index of level 2 type to int and '' if NA
i3 = multi_col.get_level_values(
    2
)  # must be null string instead of the default pd.NA or np.nan
i3 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else int(idx) for idx in i3
]  # convert index of level 2 type to int and '' if NA


multi_col = pd.MultiIndex.from_arrays([i1, i2, i3])
multi_col.names = ['tuple', 'measurement', 'step']
multi_col

multi_idx = [tuple(idx.split('_')) for idx in df.index]
multi_idx = pd.MultiIndex.from_tuples(multi_idx)
multi_idx.names = ['episodestart', 'timestamp']
multi_idx
df.columns = multi_col
df.index = multi_idx

df = df.set_index(['vehicle', 'driver', df.index])
df
# len(multi_idx)

### add rows to a partition

In [55]:

ddf = dd.read_parquet(
    'eos_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
ddf.index
ddf.npartitions
ddf.divisions
ddf.compute()

NameError: name 'dd' is not defined

In [ ]:
dfs_episode5 = dfs_episode.copy()
dfs_episode5.index = dfs_episode4.index.set_levels(
    [episodestart - pd.Timedelta(9, 'd')], level='episodestart'
)
dfs_episode5.index = dfs_episode4.index.set_levels(
    [[trucks[1].vid], [drivers[0].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
# dfs_episode5 = dfs_episode5.swaplevel(0,1, axis=0).swaplevel(1,2,axis=0)
dfs_episode5 = dfs_episode5.reset_index(level=['vehicle', 'driver'])

dfs_episode5['vehicle'] = dfs_episode5['vehicle'].astype('category')
dfs_episode5['driver'] = dfs_episode5['driver'].astype('category')
dfs_episode5

#### decoding single dataframe of multiindex into dask single index dataframe

In [56]:
df_episode_new = dfs_episode5.copy()
df_episode_new.columns = [
    f'{x[0]}_{x[1]}_{x[2]}'
    if (x[1] != '')
    else f'{x[0]}__{x[2]}'  # !!! dunder!!!
    if (x[2] != '')
    else f'{x[0]}'
    for x in dfs_episode5.columns.to_flat_index()
]
df_episode_new.index = [
    f'{x[0]}_{x[1]}' if x[1] != '' else f'x{0}'
    for x in dfs_episode5.index.to_flat_index()
]
# df_episode_new.dtypes
df_episode_new

NameError: name 'dfs_episode5' is not defined

In [ ]:

ddf = dd.read_parquet(
    'eos_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
df = ddf.compute()
df.dtypes

In [ ]:
df_list = []
for n in range(ddf.npartitions):
    df_list.append(ddf.partitions[n])
len(df_list)

In [ ]:
for i, ddf in enumerate(df_list):
    # if dfs_episode_new['vehicle']
    print(f"{i}")
    # df = ddf.compute()
    # ddf.columns
    # ddf.dtypes
    vehicle = ddf['vehicle'].compute()[0]
    driver = ddf['driver'].compute()[0]
    # df_episode_new = df_episode_new.append(ddf)
    if (
        df_episode_new['vehicle'][0] == vehicle
        and df_episode_new['driver'][0] == driver
    ):
        print(
            f'hit {i}{"st" if i==1 else "nd" if i==2 else "rd" if i==3 else "th"} partition'
        )
        df_list[i] = dd.concat([ddf, df_episode_new], axis=0)
        # append ddf to dfs_episode_new

    #     c_ep = df_episode_new['ep
    #     dfs_episode_new = dfs_episode_newappend(ddf)
    # print(f"splicing dfs_episode_new {c_ep} with


In [ ]:

ddf_all = dd.concat(df_list, axis=0, join='outer')
# ddf_all.compute()os.getcwd()
custom_metadata = {
    b'eos': b'{"length":[8, 800, 8],"timezeone":"sh","units":["kph","pct","pct"],"data_propensity":[50, 1, 4],"Dataset Description": "MP vehicle from TBox"}'
}

try:
    with ProgressBar():
        ddf_all.to_parquet(
            'eos_episodes',
            engine='pyarrow',
            compression='snappy',
            partition_on=['vehicle', 'driver'],
            write_metadata_file=True,
            custom_metadata=custom_metadata,
        )
except Exception as e:
    print(e)


In [ ]:
# ddf_all.compute()
ddf = dd.read_parquet(
    'eos_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
pq_meta = pq.read_metadata('ddf_episodes/_common_metadata')
# print(table_meta)
# print(table_meta.metadata[b'eos'])
custom_meta_info = pq_meta.metadata[b'eos']
custom_meta_info = json.loads(custom_meta_info)
# custom_meta_info

for key, val in custom_meta_info.items():
    print(f'{key}: {val}')

df = ddf.compute()

multi_tpl = [tuple(col.split('_')) for col in ddf.columns]
multi_col = pd.MultiIndex.from_tuples(multi_tpl)
i1 = multi_col.get_level_values(0)
i2 = multi_col.get_level_values(
    1
)  # must be null string instead of the default pd.NA or np.nan
i2 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i2
]  # convert index of level 2 type to int and '' if NA
i3 = multi_col.get_level_values(
    2
)  # must be null string instead of the default pd.NA or np.nan
i3 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else int(idx) for idx in i3
]  # convert index of level 2 type to int and '' if NA

multi_col = pd.MultiIndex.from_arrays([i1, i2, i3])
multi_col.names = ['tuple', 'measurement', 'step']
df.columns = multi_col

multi_idx = [tuple(idx.split('_')) for idx in ddf.index]
df_episodes_read_multiidx = df_episodes_read.copy()
multi_idx = pd.MultiIndex.from_tuples(multi_idx)
multi_idx.names = ['episodestart', 'timestamp']
df.index = multi_idx
df = df.set_index(['vehicle', 'driver', df.index])
df
df.columns
df.index
# len(multi_idx)


In [ ]:
df = df.sort_index()
df